# Uses interaction to push triples to the brain and query it

This notebook assumes you already understand the following notebooks:

* lets-chat.ipynb
* roboGrasp-api.ipynb

In this notebook, we will combine the interaction modeled through EMISSOR with the interaction throught *capsules* with the BRAIN. As auxiliary modules, we will use the *cltl-knowledgeextraction* and *cltl-language-generation*. The *cltl-knowledgeextraction* will extract triples from the utterances either for posting or for querying. In the former case, it also extracts the source perspective from the text. The triples and source perspectives are represented in enriched capsules. We will also use a replier that is include in the *cltl-languagegeneration* package. This replier transfers the response from the BRAIN into natural language expressions and possibly gestures. 

In order to connect to the EMISSOR scenario, we need to align the meta properties of the scenario with the meta data in the capsules. However,  the *cltl-knowledgeextraction* uses a similar data object to keep track of the conversation history. This is a Chat object that needs to be created. Through this Chat object, we keep track of what was said before and deal with coreference to earlier utterances.

Combining EMISSOR and the BRAIN, we can model the interaction in a infite while loop where we go through the following steps:

* We create an EMISSOR scenario at the start of the interaction and a corresponding Chat object to keep track of the dialogue history
* while the user does not stop:
    * create TextSignals for each utterance from the user and store these in an EMISSOR scenario
    * add the utterance also to the Chat instance
    * process the latest utterance in the Chat object using the *cltl-knowledgeextraction* to get triples and perspectives
    * create a capsule from the triples, perspectives and the text signal meta data
    * post the triple to the BRAIN
    * get the answer or throughts as a response from the BRAIN
    * verbalise the answer or thoughts
    * create a new TextSignal for the system response and add it to the EMISSOR scenario
* When the user stops, we save the scenario in the EMISSOR format.



Before running, start GraphDB and make sure that there is a sandbox repository.
GraphDB can be downloaded from:

https://graphdb.ontotext.com

## Import the necessary modules

In [1]:
import json
import os
import time
import uuid
from datetime import date
from datetime import datetime
from random import getrandbits, choice
import pathlib
import pprint

# general imports for EMISSOR and the BRAIN
import emissor as em
import requests
from cltl import brain
from cltl.brain.long_term_memory import LongTermMemory
from cltl.brain.utils.helper_functions import brain_response_to_json
from cltl.combot.backend.api.discrete import UtteranceType
from cltl.reply_generation.data.sentences import GREETING, ASK_NAME, ELOQUENCE, TALK_TO_ME
from cltl.reply_generation.lenka_replier import LenkaReplier
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario

/Users/piek/Desktop/t-MA-Combots-2021/code/myproject/cltl-chatbots/venv/lib/python3.7/site-packages/rdflib_jsonld/__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,
[nltk_data] Downloading package punkt to /Users/piek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Import the chatbot utility functions

In [2]:
import sys
import os

src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import chatbots.util.driver_util as d_util
import chatbots.util.capsule_util as c_util
import chatbots.intentions.talk as talk
import chatbots.intentions.get_to_know_you as friend

## Standard initialisation of a scenario

We initialise a scenario in the standard way by creating a unique folder and setting the AGENT and HUMAN_NAME and HUMAN_ID variables. Throughout this scenario, the HUMAN_NAME and HUMAN_ID will be used as the source for the utterances.

In [3]:
from random import getrandbits
import requests
##### Setting the location
place_id = getrandbits(8)
location = None
try:
    location = requests.get("https://ipinfo.io").json()
except:
    print("failed to get the IP location")
    
##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "Stranger"
HUMAN_ID = "stranger"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
# Find the repository root dir
parent, dir_name = (d_util.__file__, "_")
while dir_name and dir_name != "src":
    parent, dir_name = os.path.split(parent)
root_dir = parent
scenario_path = os.path.abspath(os.path.join(root_dir, 'data'))

if not os.path.exists(scenario_path) :
    os.mkdir(scenario_path)
    print("Created a data folder for storing the scenarios", scenario_path)

### Define the folders where the images and rdf triples are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"
rdffolder = scenario_path + "/" + scenario_id + "/" + "rdf"

### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario_ctrl = scenarioStorage.create_scenario(scenario_id, int(time.time() * 1e3), None, AGENT)

Directories for 2021-12-02-18:18:21 created in /Users/piek/Desktop/t-MA-Combots-2021/code/myproject/cltl-chatbots/data


## Specifying the BRAIN

We specify the BRAIN in GraphDB and use the scenario path just defined for storing the RDF triple produced in EMISSOR.

If you set *clear_all* to *True*, the sandbox triple store is emptied (memory erased) and the basic ontological models are reloaded. Setting it to *False* means you add things to the current memory.

In [4]:
log_path = pathlib.Path(rdffolder)
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                                log_dir=log_path,
                                clear_all=True)


2021-12-02 18:18:24,044 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Booted
2021-12-02 18:18:25,649 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-12-02 18:18:26,970 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Booted
2021-12-02 18:18:26,972 -     INFO -  cltl.brain.basic_brain.LocationReasoner - Booted
2021-12-02 18:18:26,973 -     INFO -      cltl.brain.basic_brain.TypeReasoner - Booted
2021-12-02 18:18:26,975 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Booted
2021-12-02 18:18:27,086 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents


## Create an instance of a replier

In [5]:
replier = LenkaReplier()

2021-12-02 18:18:29,342 -     INFO -   cltl.reply_generation.api.LenkaReplier - Booted


## Establish the speaker identity

In [6]:
#### Small sequence to learn name of speaker
initial_prompt = f"{choice(GREETING)} {choice(ASK_NAME)} {HUMAN_NAME}?"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario_ctrl, initial_prompt)
scenario_ctrl.append_signal(textSignal)

#### Get name from person 
HUMAN_NAME, HUMAN_ID = friend.get_a_name_and_id(scenario_ctrl, AGENT)
HUMAN_ID = HUMAN_NAME  ### Hack because we cannot force the namespace through capsules, name and identity are the same till this is fixed

print("Name:", HUMAN_NAME)
print("Id:", HUMAN_ID)
                
capsule = c_util.scenario_utterance_to_capsule(scenario_ctrl,place_id,location, textSignal,HUMAN_ID,AGENT,"know", HUMAN_ID)

name_thoughts = my_brain.update(capsule, reason_types=True, create_label=True)

#pprint.pprint(capsule)

Leolani2: How are you doing? What is your name? Stranger?



 Piek


Piek
Leolani2: So your name is Piek?



 yes


Name: Piek
Id: Piek
2021-12-02 18:18:39,354 -  WARNING -      cltl.brain.basic_brain.TypeReasoner - Failed to query Wikidata: HTTPSConnectionPool(host='query.wikidata.org', port=443): Read timed out. (read timeout=3)
2021-12-02 18:18:40,028 -     INFO -      cltl.brain.basic_brain.TypeReasoner - Reasoned type of Piek to: None
2021-12-02 18:18:40,153 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: leolani2_know_piek [person_->_])
2021-12-02 18:18:40,205 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: new subject - new object 
2021-12-02 18:18:44,408 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Negation Conflicts: piek on December,2021 about UNDERSPECIFIED
2021-12-02 18:18:44,458 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. like-by agent - 15 gaps as object: e.g. dislike agent
2021-12-02 18:18:44,548 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 25 gaps as subject: e.

## Initialise a chat with the HUMAN_ID to keep track of the dialogue history

In [7]:
chat = Chat(HUMAN_ID)

2021-12-02 18:18:44,558 -     INFO - cltl.triple_extraction.api.Chat (Piek)              000 - << Start of Chat with Piek >>


## Start the interaction

In [ ]:
print_details=False


#### Initial prompt by the system from which we create a TextSignal and store it
initial_prompt = f"{choice(TALK_TO_ME)}"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario_ctrl, initial_prompt)
scenario_ctrl.append_signal(textSignal)

utterance = ""
#### Get input and loop
while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    ###### Getting the next input signals
    utterance = input('\n')
    print(HUMAN_NAME + ": " + utterance)
    textSignal = d_util.create_text_signal(scenario_ctrl, utterance)
    scenario_ctrl.append_signal(textSignal)

    #### Process input and generate reply
    
    capsule, reply = talk.process_text_and_reply(scenario_ctrl,
                           place_id,
                           location,
                           HUMAN_ID,
                           textSignal,
                           chat,
                           replier,
                           my_brain,
                           print_details)

    print(AGENT + ": " + reply)
    textSignal = d_util.create_text_signal(scenario_ctrl, reply)
    scenario_ctrl.append_signal(textSignal)


Leolani2: Do you have any gossip?



 where are at apples


Piek: where are at apples
2021-12-02 18:34:00,817 -     INFO - cltl.triple_extraction.api.Chat (Piek)              014 -       Piek: "where are at apples"
DEBUG tree (U
  (Q
    (W (WRB where))
    (VP (V (VBP are)))
    (C (NP (PREP (IN at)) (NP (N (NNS apples)))))))
DEBUG predicate are
DEBUG label VBP
DEBUG before object analysis: {'predicate': 'be', 'subject': 'at-apples', 'object': ''}
DEBUG after object analysis: {'predicate': 'be', 'subject': 'at-apples', 'object': ''}
DEBUG before subject analysis: {'predicate': 'be', 'subject': 'at-apples', 'object': ''}
DEBUG triple subject preposition at-apples
DEBUG after subject analysis: {'predicate': 'be-at', 'subject': 'apples', 'object': ''}
DEBUG final triple after fix: {'predicate': {'label': 'be-at', 'type': ['verb.stative', 'prep']}, 'subject': {'label': 'apples', 'type': ['noun.food']}, 'object': {'label': '', 'type': []}} {'neg': False, 'wh_word': {'response': 'location'}}
2021-12-02 18:34:00,825 -     INFO -    cltl.brain.basic_b


 where are under apples


Piek: where are under apples
2021-12-02 18:34:15,799 -     INFO - cltl.triple_extraction.api.Chat (Piek)              015 -       Piek: "where are under apples"
DEBUG tree (U
  (Q
    (W (WRB where))
    (VP (V (VBP are)))
    (C (NP (PREP (IN under)) (NP (N (NNS apples)))))))
DEBUG predicate are
DEBUG label VBP
DEBUG before object analysis: {'predicate': 'be', 'subject': 'under-apples', 'object': ''}
DEBUG after object analysis: {'predicate': 'be', 'subject': 'under-apples', 'object': ''}
DEBUG before subject analysis: {'predicate': 'be', 'subject': 'under-apples', 'object': ''}
DEBUG triple subject preposition under-apples
DEBUG after subject analysis: {'predicate': 'be-under', 'subject': 'apples', 'object': ''}
DEBUG final triple after fix: {'predicate': {'label': 'be-under', 'type': ['verb.stative', 'prep']}, 'subject': {'label': 'apples', 'type': ['noun.food']}, 'object': {'label': '', 'type': []}} {'neg': False, 'wh_word': {'response': 'location'}}
2021-12-02 18:34:15,803 -     I

### Adapt the ontology with new properties and classes

In [11]:
my_brain.ontology_graph.parse(location=os.path.join('/Users/piek/Desktop/t-MA-Combots-2021/code/myproject', 'test.ttl'), format="turtle")

<Graph identifier=http://cltl.nl/leolani/world/Ontology (<class 'rdflib.graph.Graph'>)>

## Save the scenario data

In [9]:
scenario_ctrl.scenario.ruler.end = int(time.time() * 1e3)
scenarioStorage.save_scenario(scenario_ctrl)

## End of notebook